In [ ]:
class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()
        self.conv_1 = 

    def forward(self, x,....):
        k = self.conv_1(x)




        return k 


I've modified the architecture into a node-level predictor: it now outputs a score per atom indicating the likelihood of being the borylation site. This version replaces the graph-level readout with a node_predictor head suitable for binary classification per node.

In [ ]:
import torch
import torch.nn as nn
from dgl.nn.pytorch import NNConv

class MPNN_NodeClassifier(nn.Module):
    def __init__(self, node_in_feats, edge_in_feats, hidden_feats=64, num_step_message_passing=3):
        super(MPNN_NodeClassifier, self).__init__()

        self.project_node_feats = nn.Sequential(
            nn.Linear(node_in_feats, hidden_feats),
            nn.ReLU()
        )

        edge_network = nn.Linear(edge_in_feats, hidden_feats * hidden_feats)

        self.gnn_layer = NNConv(
            in_feats=hidden_feats,
            out_feats=hidden_feats,
            edge_func=edge_network,
            aggregator_type='sum'
        )

        self.activation = nn.ReLU()
        self.gru = nn.GRU(hidden_feats, hidden_feats)

        self.node_predictor = nn.Sequential(
            nn.Linear(hidden_feats, hidden_feats),
            nn.ReLU(),
            nn.Linear(hidden_feats, 1)  # Predicting a binary label: is borylation site or not
        )

        self.num_step_message_passing = num_step_message_passing

    def forward(self, g):
        node_feats = g.ndata['attr']  # shape: [num_nodes, node_in_feats]
        edge_feats = g.edata['edge_attr']

        node_feats = self.project_node_feats(node_feats)  # shape: [num_nodes, hidden_feats]
        hidden_feats = node_feats.unsqueeze(0)  # shape: [1, num_nodes, hidden_feats]

        for _ in range(self.num_step_message_passing):
            node_feats_new = self.activation(self.gnn_layer(g, node_feats, edge_feats)).unsqueeze(0)
            node_feats, hidden_feats = self.gru(node_feats_new, hidden_feats)
            node_feats = node_feats.squeeze(0)

        g.ndata['node_logits'] = self.node_predictor(node_feats).squeeze(-1)  # shape: [num_nodes]
        return g.ndata['node_logits']


# Example usage:
# model = MPNN_NodeClassifier(node_in_feats=ATOM_FEATURE_DIM, edge_in_feats=BOND_FEATURE_DIM)
# logits = model(graph)
# Apply torch.sigmoid(logits) if using BCEWithLogitsLoss during training
